In [4]:
import os
import argparse
from itertools import chain
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy
from progressbar import ETA, Bar, Percentage, ProgressBar
import cv2
import numpy as np
import pandas as pd
from scipy import resize
import scipy.io
import ipdb

## Dataset Manipulation

In [12]:
dataset_path = '../datasets/'
handbag_path = os.path.join(dataset_path, 'edges2handbags')
shoe_path = os.path.join(dataset_path, 'edges2shoes')

def shuffle_data(da, db):
    a_in = list(range(len(da)))
    np.random.shuffle(a_in)
    
    b_in = list(range(len(db)))
    np.random.shuffle(b_in)
    
    shuffled_da = np.array(da)[np.array(a_in)]
    shuffled_db = np.array(db)[np.array(b_in)]
    
    return shuffled_da, shuffled_db

def read_images(filenames, domain=None, image_size=64):
#     print(filenames)
    images = []
    for fn in filenames:
#         print(fn)
        image = cv2.imread(fn)
        if image is None:
            continue
        if domain == 'A':
            kernel = np.ones((3,3), np.uint8)
            image = image[:, :256, :]
            image = 255. - image
            image = cv2.dilate(image, kernel, iterations=1)
            image = 255. - image
        elif domain == 'B':
            image = image[:, 256:, :]
            
        image = cv2.resize(image, (image_size, image_size))
        image = image.astype(np.float32) / 255.
        image = image.transpose(2,0,1)
        images.append(image)
        
    images = np.stack(images)
    return images

def read_attr(attr_path, image_dir):
    f = open(attr_path)
    lines = f.readlines()
    lines = map(lambda line: line.strip(), lines)
    columns = ['image_path'] + lines[1].split()
    lines = lines[2:]
    
    items = map(lambda line: line.split(), lines)
    df = pd.DataFrame(item, columns=columns)
    df['image_path'] = df['image_path'].map( lambda x: os.path.join( image_dir, x ) )

    return df

def get_edge2photo_files(item='edges2handbags', test=False):
    if item == 'edges2handbags':
        item_path = handbag_path
    elif item == 'edges2shoes':
        item_path = shoe_path

    if test == True:
        item_path = os.path.join( item_path, 'val' )
    else:
        item_path = os.path.join( item_path, 'train' )

    image_paths = map(lambda x: os.path.join( item_path, x ), os.listdir( item_path ))
    
#     print (image_paths)
    image_paths = list(image_paths)
#     print (image_paths)
    if test == True:
        return [image_paths, image_paths]
    else:
        n_images = len(image_paths)
        return [image_paths[:int(n_images/2)], image_paths[int(n_images/2):]]

## Model

In [16]:
kernel_sizes = [4,3,3]
strides = [2,2,1]
paddings = [0,0,1]

latent_dim = 300


class Discriminator(nn.Module):
    def __init__(self,):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 4, 2, 1, bias=False)
        self.relu1 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv2 = nn.Conv2d(64, 2*64, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(64*2)
        self.relu2 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv3 = nn.Conv2d(64*2, 64*4, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(64*4)
        self.relu3 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv4 = nn.Conv2d(64*4, 64*8, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(64*8)
        self.relu4 = nn.LeakyReLU(0.2, inplace=True)
        
        self.conv5 = nn.Conv2d(64*8, 1, 4, 1, 0, bias=False)
        
    def forward(self, input):
        conv1 = self.conv1(input)
        relu1 = self.relu1(conv1)
        
        conv2 = self.conv2(relu1)
        bn2 = self.bn2(conv2)
        relu2 = self.relu2(bn2)
        
        conv3 = self.conv3(relu2)
        bn3 = self.bn3(conv3)
        relu3 = self.relu3(bn3)
        
        conv4 = self.conv4(relu3)
        bn4 = self.bn4(conv4)
        relu4 = self.relu4(bn4)
        
        conv5 = self.conv5(relu4)
        
        return torch.sigmoid(conv5), [relu2,relu3, relu4]
    
class Generator(nn.Module):
    def __init__(self, extra_layers=False):
        super(Generator, self).__init__()
        if extra_layers == True:
            self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 8, 100, 4, 1, 0, bias=False),
                nn.BatchNorm2d(100),
                nn.LeakyReLU(0.2, inplace=True),
                
                nn.ConvTranspose2d(100, 64 * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 2,     64, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(    64,      3, 4, 2, 1, bias=False),
                nn.Sigmoid()
            )
            
        if extra_layers == False:
            self.main = nn.Sequential(
                nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 8),
                nn.LeakyReLU(0.2, inplace=True),

                nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64 * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(64 * 2,     64, 4, 2, 1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True),
                nn.ConvTranspose2d(    64,      3, 4, 2, 1, bias=False),
                nn.Sigmoid()
            )
        
    def forward(self, input):
        return self.main(input)
        
#         super(GeneratorCNN, self).__init__()
#         self.num_gpu = num_gpu
#         self.layers = []

#         prev_dim = conv_dims[0]
#         self.layers.append(nn.Conv2d(input_channel, prev_dim, 4, 2, 1, bias=False))
#         self.layers.append(nn.LeakyReLU(0.2, inplace=True))

#         for out_dim in conv_dims[1:]:
#             self.layers.append(nn.Conv2d(prev_dim, out_dim, 4, 2, 1, bias=False))
#             self.layers.append(nn.BatchNorm2d(out_dim))
#             self.layers.append(nn.LeakyReLU(0.2, inplace=True))
#             prev_dim = out_dim

#         for out_dim in deconv_dims:
#             self.layers.append(nn.ConvTranspose2d(prev_dim, out_dim, 4, 2, 1, bias=False))
#             self.layers.append(nn.BatchNorm2d(out_dim))
#             self.layers.append(nn.ReLU(True))
#             prev_dim = out_dim

#         self.layers.append(nn.ConvTranspose2d(prev_dim, output_channel, 4, 2, 1, bias=False))
#         self.layers.append(nn.Tanh())

#         self.layer_module = nn.ModuleList(self.layers)

#     def main(self, x):
#         out = x
#         for layer in self.layer_module:
#             out = layer(out)
#         return out

#     def forward(self, x):
#         return self.main(x)


## Helper

In [7]:
def as_np(data):
    return data.cpu().data.numpy()

def get_data():
    data_A_1, data_A_2 = get_edge2photo_files( item='edges2handbags', test=False )
    test_A_1, test_A_2 = get_edge2photo_files( item='edges2handbags', test=True )

    data_A = np.hstack( [data_A_1, data_A_2] )
    test_A = np.hstack( [test_A_1, test_A_2] )

    data_B_1, data_B_2 = get_edge2photo_files( item='edges2shoes', test=False )
    test_B_1, test_B_2 = get_edge2photo_files( item='edges2shoes', test=True )

    data_B = np.hstack( [data_B_1, data_B_2] )
    test_B = np.hstack( [test_B_1, test_B_2] )

    
    return data_A, data_B, test_A, test_B

def get_fm_loss(real_feats, fake_feats, criterion):
    losses = 0
    for real_feat, fake_feat in zip(real_feats, fake_feats):
        l2 = (real_feat.mean(0) - fake_feat.mean(0)) * (real_feat.mean(0) - fake_feat.mean(0))
        loss = criterion( l2, Variable( torch.ones( l2.size() ) ).cuda() )
        losses += loss

    return losses

def get_gan_loss(dis_real, dis_fake, criterion, cuda):
    labels_dis_real = Variable(torch.ones( [dis_real.size()[0], 1] ))
    labels_dis_fake = Variable(torch.zeros([dis_fake.size()[0], 1] ))
    labels_gen = Variable(torch.ones([dis_fake.size()[0], 1]))

    if cuda:
        labels_dis_real = labels_dis_real.cuda()
        labels_dis_fake = labels_dis_fake.cuda()
        labels_gen = labels_gen.cuda()

    dis_loss = criterion( dis_real, labels_dis_real ) * 0.5 + criterion( dis_fake, labels_dis_fake ) * 0.5
    gen_loss = criterion( dis_fake, labels_gen )

    return dis_loss, gen_loss
    

## Driver

In [ ]:
cuda = True
task_name = 'handbags2shoes'

epoch_size = 5000
batch_size = 64

result_path = os.path.join('./results/', task_name)
model_path = os.path.join('./models/', task_name)

data_style_A, data_style_B, test_style_A, test_style_B = get_data()
test_A = read_images( test_style_A, 'B')
test_B = read_images( test_style_B, 'B')


test_A = Variable( torch.FloatTensor( test_A ), volatile=True )
test_B = Variable( torch.FloatTensor( test_B ), volatile=True )

if not os.path.exists(result_path):
    os.makedirs(result_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)

generator_A = Generator()
generator_B = Generator()
discriminator_A = Discriminator()
discriminator_B = Discriminator()

if cuda:
    test_A = test_A.cuda()
    test_B = test_B.cuda()
    generator_A = generator_A.cuda()
    generator_B = generator_B.cuda()
    discriminator_A = discriminator_A.cuda()
    discriminator_B = discriminator_B.cuda()
    
data_size = min( len(data_style_A), len(data_style_B) )
n_batches = ( data_size // batch_size )

recon_criterion = nn.MSELoss()
gan_criterion = nn.BCELoss()
feat_criterion = nn.HingeEmbeddingLoss()

gen_params = chain(generator_A.parameters(), generator_B.parameters())
dis_params = chain(discriminator_A.parameters(), discriminator_B.parameters())

optim_gen = optim.Adam( gen_params, lr=0.0002, betas=(0.5,0.999), weight_decay=0.00001)
optim_dis = optim.Adam( dis_params, lr=0.0002, betas=(0.5,0.999), weight_decay=0.00001)

iters = 0

gen_loss_total = []
dis_loss_total = []

for epoch in range(epoch_size):
    
    data_style_A, data_style_B = shuffle_data( data_style_A, data_style_B)

    widgets = ['epoch #%d|' % epoch, Percentage(), Bar(), ETA()]
    pbar = ProgressBar(maxval=n_batches, widgets=widgets)
    pbar.start()

    for i in range(n_batches):

        pbar.update(i)

        generator_A.zero_grad()
        generator_B.zero_grad()
        discriminator_A.zero_grad()
        discriminator_B.zero_grad()

        A_path = data_style_A[ i * batch_size: (i+1) * batch_size ]
        B_path = data_style_B[ i * batch_size: (i+1) * batch_size ]

        
        A = read_images( A_path, 'B', 64 )
        B = read_images( B_path, 'B',  64 )

        A = Variable( torch.FloatTensor( A ) )
        B = Variable( torch.FloatTensor( B ) )

        if cuda:
            A = A.cuda()
            B = B.cuda()

        AB = generator_B(A)
        BA = generator_A(B)

        ABA = generator_A(AB)
        BAB = generator_B(BA)

        # Reconstruction Loss
        recon_loss_A = recon_criterion( ABA, A )
        recon_loss_B = recon_criterion( BAB, B )

        # Real/Fake GAN Loss (A)
        A_dis_real, A_feats_real = discriminator_A( A )
        A_dis_fake, A_feats_fake = discriminator_A( BA )

        dis_loss_A, gen_loss_A = get_gan_loss( A_dis_real, A_dis_fake, gan_criterion, cuda )
        fm_loss_A = get_fm_loss(A_feats_real, A_feats_fake, feat_criterion)

        # Real/Fake GAN Loss (B)
        B_dis_real, B_feats_real = discriminator_B( B )
        B_dis_fake, B_feats_fake = discriminator_B( AB )

        dis_loss_B, gen_loss_B = get_gan_loss( B_dis_real, B_dis_fake, gan_criterion, cuda )
        fm_loss_B = get_fm_loss( B_feats_real, B_feats_fake, feat_criterion )

        # Total Loss

        if iters < 1000:
            rate = 0.5
        else:
            rate = 0.5

        gen_loss_A_total = (gen_loss_B*0.1 + fm_loss_B*0.9) * (1.-rate) + recon_loss_A * rate
        gen_loss_B_total = (gen_loss_A*0.1 + fm_loss_A*0.9) * (1.-rate) + recon_loss_B * rate

        gen_loss = gen_loss_A_total + gen_loss_B_total
        dis_loss = dis_loss_A + dis_loss_B

        if iters % 3 == 0:
            dis_loss.backward()
            optim_dis.step()
        else:
            gen_loss.backward()
            optim_gen.step()

        if iters % 50 == 0:
            print ("---------------------")
            print ("GEN Loss:", as_np(gen_loss_A.mean()), as_np(gen_loss_B.mean()))
            print ("Feature Matching Loss:", as_np(fm_loss_A.mean()), as_np(fm_loss_B.mean()))
            print ("RECON Loss:", as_np(recon_loss_A.mean()), as_np(recon_loss_B.mean()))
            print ("DIS Loss:", as_np(dis_loss_A.mean()), as_np(dis_loss_B.mean()))

        if iters % 1000 == 0:
            AB = generator_B( test_A )
            BA = generator_A( test_B )
            ABA = generator_A( AB )
            BAB = generator_B( BA )

            n_testset = min( test_A.size()[0], test_B.size()[0] )

            subdir_path = os.path.join( result_path, str(iters / 1000) )

            if os.path.exists( subdir_path ):
                pass
            else:
                os.makedirs( subdir_path )

            for im_idx in range( n_testset ):
                A_val = test_A[im_idx].cpu().data.numpy().transpose(1,2,0) * 255.
                B_val = test_B[im_idx].cpu().data.numpy().transpose(1,2,0) * 255.
                BA_val = BA[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
                ABA_val = ABA[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
                AB_val = AB[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.
                BAB_val = BAB[im_idx].cpu().data.numpy().transpose(1,2,0)* 255.

                filename_prefix = os.path.join (subdir_path, str(1))
                scipy.misc.imsave( filename_prefix + '.A.jpg', A_val.astype(np.uint8)[:,:,::-1])
                scipy.misc.imsave( filename_prefix + '.B.jpg', B_val.astype(np.uint8)[:,:,::-1])
                scipy.misc.imsave( filename_prefix + '.BA.jpg', BA_val.astype(np.uint8)[:,:,::-1])
                scipy.misc.imsave( filename_prefix + '.AB.jpg', AB_val.astype(np.uint8)[:,:,::-1])
                scipy.misc.imsave( filename_prefix + '.ABA.jpg', ABA_val.astype(np.uint8)[:,:,::-1])
                scipy.misc.imsave( filename_prefix + '.BAB.jpg', BAB_val.astype(np.uint8)[:,:,::-1])

        if iters % 10000 == 0:
            torch.save( generator_A, os.path.join(model_path, 'model_gen_A-' + str( iters / 10000 )))
            torch.save( generator_B, os.path.join(model_path, 'model_gen_B-' + str( iters / 10000 )))
            torch.save( discriminator_A, os.path.join(model_path, 'model_dis_A-' + str( iters / 10000 )))
            torch.save( discriminator_B, os.path.join(model_path, 'model_dis_B-' + str( iters / 10000 )))

        iters += 1

/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


---------------------
GEN Loss: 0.68547785 0.60867894
Feature Matching Loss: 0.18831362 0.1691957
RECON Loss: 0.17518532 0.20073049
DIS Loss: 0.6554173 0.7332457


/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:160: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:161: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:162: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:163: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/opt/anaconda3/envs/tengpu/lib/python3.6/site-packages/ipykernel_launcher.py:164: Deprec

---------------------
GEN Loss: 4.1272526 1.2591856
Feature Matching Loss: 0.030108843 0.028684862
RECON Loss: 0.12977698 0.14205839
DIS Loss: 1.1856214 0.62592477


---------------------
GEN Loss: 1.3293315 1.0535858
Feature Matching Loss: 0.03770527 0.018339686
RECON Loss: 0.11474195 0.11863279
DIS Loss: 0.43238777 0.5302632


...

---------------------
GEN Loss: 0.7609939 2.1359432
Feature Matching Loss: 0.00964787 0.00820012
RECON Loss: 0.042426992 0.026181662
DIS Loss: 0.48117375 0.35713696


---------------------
GEN Loss: 1.6610024 1.2445916
Feature Matching Loss: 0.013530852 0.010167123
RECON Loss: 0.0393683 0.028957376
DIS Loss: 0.5540294 0.30261484


---------------------
GEN Loss: 1.5799232 2.0390034
Feature Matching Loss: 0.008651014 0.011274502
RECON Loss: 0.039234 0.028805107
DIS Loss: 0.5309372 0.1793796


---------------------
GEN Loss: 2.6613388 2.9253073
Feature Matching Loss: 0.010639087 0.008913469
RECON Loss: 0.04267605 0.028575242
DIS Loss: 0.4485647 0.4360898


---------------------
GEN Loss: 0.8805704 2.8503118
Feature Matching Loss: 0.008633911 0.010812323
RECON Loss: 0.03378981 0.02714631
DIS Loss: 0.67434037 0.281803


In [13]:
torch._C._cuda_getDriverVersion()

10000

In [14]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [19]:
cuda

True